In [2]:
import pandas as pd
import scipy.optimize as opt
import numpy as np

In [46]:
# read in demand data
demand = pd.read_csv("/content/melons_demand.csv")

# read in supply predictions data
actual_supply = pd.read_csv("/content/melons_var_predict.csv")

# read in prices data
wholesale_prices = pd.read_csv("/content/wholesale_prices_short.csv")
retail_prices = pd.read_csv("/content/est_retail_prices.csv")

In [3]:
wholesale_prices = pd.read_csv("/content/wholesale_prices_short.csv")
retail_prices = pd.read_csv("/content/est_retail_prices.csv")

In [47]:
demand = demand[demand["year"] >= 1998]
demand = demand[demand["year"] <= 2015]
demand.reset_index(inplace=True, drop=True)

demand.head()

,year,Chicago,Detroit,Minneapolis.StPaul,Boston,New.York,Philadelphia,Atlanta,Baltimore,Dallas.FortWorth,Houston,Miami,DC,Los.Angeles,San.Francisco,San.Diego,Seattle,Average
0,1998,2.436183e+08,1.120668e+08,8.320517e+07,1.039235e+08,5.693159e+08,1.155111e+08,7.804786e+07,5.790986e+07,1.282182e+08,1.004015e+08,1.056220e+08,1.419328e+08,2.992232e+08,1.680264e+08,7.660786e+07,9.034500e+07,1.455280e+08
1,1999,2.478301e+08,1.210409e+08,8.846021e+07,1.283255e+08,5.870718e+08,1.432932e+08,9.207665e+07,7.077088e+07,1.621069e+08,1.195715e+08,1.308010e+08,1.534263e+08,3.418262e+08,1.802055e+08,8.205053e+07,1.074186e+08,1.621339e+08
2,2000,2.399220e+08,1.334454e+08,9.988364e+07,1.335212e+08,7.060391e+08,1.798704e+08,1.028864e+08,7.320460e+07,1.964476e+08,1.308461e+08,1.633857e+08,1.709104e+08,3.846621e+08,1.904372e+08,7.761127e+07,1.355978e+08,1.834512e+08
3,2001,2.776088e+08,1.287718e+08,1.031443e+08,1.408708e+08,7.218144e+08,1.846595e+08,1.002539e+08,7.051424e+07,2.019659e+08,1.277420e+08,1.695805e+08,1.583793e+08,3.692796e+08,1.789070e+08,6.990928e+07,1.369644e+08,1.847274e+08
4,2002,2.691112e+08,1.200549e+08,1.040898e+08,1.467602e+08,6.718729e+08,1.481479e+08,1.133728e+08,7.701803e+07,1.793606e+08,1.251406e+08,1.632516e+08,1.637959e+08,3.782997e+08,1.617474e+08,7.547100e+07,1.244447e+08,1.777611e+08


In [48]:
actual_supply = actual_supply[actual_supply["year"] >= 1999]
actual_supply.reset_index(inplace=True, drop=True)

actual_supply.head()

,year,west_lbs,p_sw_lbs,mw_se_lbs,a_ne_lbs,west_half_lbs,east_half_lbs
0,1999,1.044025e+09,5.907323e+08,8.092691e+08,6.274396e+07,1.634758e+09,8.720130e+08
1,2000,1.187345e+09,6.450225e+08,9.979128e+08,6.433720e+07,1.832367e+09,1.062250e+09
2,2001,1.067281e+09,6.730230e+08,8.420110e+08,6.314855e+07,1.740304e+09,9.051595e+08
3,2002,1.072191e+09,7.499328e+08,8.200027e+08,6.818165e+07,1.822123e+09,8.881843e+08
4,2003,1.067864e+09,7.075389e+08,8.402313e+08,6.774338e+07,1.775403e+09,9.079747e+08


In [49]:
wholesale_prices = wholesale_prices[wholesale_prices["year"] >= 1999]
wholesale_prices = wholesale_prices[wholesale_prices["year"] <= 2015]
wholesale_prices.reset_index(inplace=True, drop=True)

wholesale_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,21.3,35.0,33.00,28.6,74.4,102,17.250,11.101633,67.161655,...,16.8,24.1,32.1,15.098916,46.5,11.3,9.7,16.9,27.428610,6.9
1,2000,17.8,36.0,32.35,27.6,64.9,115,7.000,11.035726,79.001992,...,13.1,31.2,31.8,19.228349,42.6,12.3,11.2,18.5,28.129329,5.3
2,2001,22.9,38.0,34.50,30.7,75.8,109,9.625,11.863701,72.454395,...,17.1,26.5,32.4,19.287165,45.0,13.4,10.7,19.5,27.847922,10.8
3,2002,25.8,43.0,34.30,30.6,71.3,117,11.250,12.184179,87.104935,...,19.1,31.4,34.2,23.559195,47.6,12.9,12.1,19.2,27.546230,9.6
4,2003,26.1,41.0,34.45,29.1,74.9,120,8.125,12.388878,79.634487,...,19.0,32.7,36.9,21.827844,49.2,13.3,13.7,19.2,28.384431,7.3


In [50]:
retail_prices = retail_prices[retail_prices["year"] >= 1999]
retail_prices = retail_prices[retail_prices["year"] <= 2015]
retail_prices.reset_index(inplace=True, drop=True)

retail_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,86.0,52.5,49.500,42.90,208,153.0,25.8750,16.652449,100.742482,...,25.20,36.15,48.15,22.648374,69.75,16.95,14.55,25.35,41.142914,10.35
1,2000,88.0,54.0,48.525,41.40,188,172.5,10.5000,16.553589,118.502987,...,19.65,46.80,47.70,28.842524,63.90,18.45,16.80,27.75,42.193994,7.95
2,2001,83.0,57.0,51.750,46.05,229,163.5,14.4375,17.795551,108.681593,...,25.65,39.75,48.60,28.930747,67.50,20.10,16.05,29.25,41.771883,16.20
3,2002,91.0,64.5,51.450,45.90,228,175.5,16.8750,18.276268,130.657402,...,28.65,47.10,51.30,35.338792,71.40,19.35,18.15,28.80,41.319345,14.40
4,2003,94.0,61.5,51.675,43.65,237,180.0,12.1875,18.583316,119.451730,...,28.50,49.05,55.35,32.741766,73.80,19.95,20.55,28.80,42.576646,10.95


In [51]:
# parameters
N = 16 # number of retailers
K = demand.shape[0]-1 # number of iterations (0th one is the initial condition)

# dataframes to keep track of orders
# columns represent retailers, rows represent iterations
orders_west = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_east = pd.DataFrame(columns=range(0,N), index=range(0,K+1))

# initial orders
for i in range(0,N):
  orders_west.iloc[0,i] = demand.iloc[0,i+1] / 2
  orders_east.iloc[0,i] = demand.iloc[0,i+1] / 2

In [52]:
def ret_cost_sim(q, k, curr_i, orders_west, orders_east, demand, supply, wholesale_prices, retail_prices):
  Q_i = demand.iloc[k+1,curr_i+1]

  total_request_west = q[0]
  total_request_west += orders_west.iloc[k,0:curr_i].sum() + orders_west.iloc[k,curr_i+1:].sum()

  total_request_east = q[1]
  total_request_east += orders_east.iloc[k,0:curr_i].sum() + orders_east.iloc[k,curr_i+1:].sum()

  if (total_request_west <= supply["west_half_lbs"][k]):
    curr_R_west = q[0]
  else:
    curr_R_west = supply["west_half_lbs"][k] * (q[0] / total_request_west)

  if (total_request_east <= supply["east_half_lbs"][k]):
    curr_R_east = q[1]
  else:
    curr_R_east = supply["east_half_lbs"][k] * (q[1] / total_request_east)

  R = curr_R_west + curr_R_east

  holding = max(R - Q_i, 0)
  stockout = max(Q_i - R, 0)

  holding = wholesale_prices["Melons"][k] * holding
  stockout = retail_prices["Melons"][k] * stockout

  cost = holding + stockout
  return cost

In [53]:
for k in range(0,K):
  print(k)

  k_opt_west = np.zeros(N)
  k_opt_east = np.zeros(N)

  for curr_i in range(0,N):
    rranges = (slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5))
    params_curr = (k, curr_i, orders_west, orders_east, demand, actual_supply, wholesale_prices, retail_prices)
    opt_sol = opt.brute(ret_cost_sim, rranges, args=params_curr, full_output=True)

    if opt_sol[0][0] > 0:
      orders_west.iloc[k+1,curr_i] = opt_sol[0][0]
    else:
      orders_west.iloc[k+1,curr_i] = 0

    if opt_sol[0][1] > 0:
      orders_east.iloc[k+1,curr_i] = opt_sol[0][1]
    else:
      orders_east.iloc[k+1,curr_i] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [54]:
orders_total = orders_west + orders_east
(orders_total.iloc[:,0] - demand["Chicago"]) / 10**7

0          0.0
1     1.010836
2     0.704639
3     3.153296
4     1.497075
5     2.846003
6     1.154024
7     2.284313
8     1.045459
9     1.844319
10    0.713805
11    1.760403
12    1.193346
13    1.985144
14    1.469831
15    2.397882
16    1.388706
17    2.242599
dtype: object

In [55]:
orders_west.to_csv("/content/melons_nonid_var_orders_west.csv", index=False)
orders_east.to_csv("/content/melons_nonid_var_orders_east.csv", index=False)